In [ ]:
%matplotlib inline

import serial
import struct
import sys
import binascii
import inspect
import time
import math
import h5py
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
import numpy as np
from collections import deque

import quaternion
import madgwickahrs
import skeleton

#Object dectection Yolo CNN
import cv2
import sys
sys.path.insert(0, '/home/rahulsingh/Desktop/earble/yolo')
from darkflow.net.build import TFNet

options = {"model": "cfg/yolo.cfg", "load": "yolo/bin/yolo.weights", "threshold": 0.2}
tfnet = TFNet(options)

#Ipython specific commands
from IPython import display


##########################################################################################################################
##Some functions on data preprocessing are also required
##command 'exec()' can be used
######$$ Take care of the disconnection event which occurs in between and make sure all the bles are connected all the time
##disconnect in between ==> so reconnect and display disconnection msg
#attributes can be written
#or stack time stamp before
#There is some issue with Magnetometer data as it seems to be 16 bit --> in a way acc and gyro seem to be 8-bit but accordign to benny
#they are 16 bit bit integers
##########################################################################################################################

## HELPFUL SITES
#Appending and popping from a list of lists/ array of lists
#1. https://stackoverflow.com/questions/33983053/how-to-create-a-numpy-array-of-lists
#2. https://stackoverflow.com/questions/8713620/appending-items-to-a-list-of-lists-in-python

#8-bit unsigned to signed conversion
#https://stackoverflow.com/questions/17067813/convert-ascii-character-to-signed-8-bit-integer-python
#https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.dtypes.html
#Numpy slicing: https://stackoverflow.com/questions/25876640/subsampling-every-nth-entry-in-a-numpy-array
#list to array: https://stackoverflow.com/questions/10346336/list-of-lists-into-numpy-array
#More on python arrays: http://pages.physics.cornell.edu/~myers/teaching/ComputationalMethods/python/arrays.html
#Resizable h5py dataset: http://docs.h5py.org/en/2.3/high/dataset.html#resizable-datasets
#Extracting array column: https://stackoverflow.com/questions/8386675/extracting-specific-columns-in-numpy-array

##IMU Related site
#1. IMU Lib: https://github.com/richardstechnotes/RTIMULib2/tree/master/Linux/python
#2. Forum: https://www.raspberrypi.org/forums/viewtopic.php?t=127930
#https://jeanleflambeur.wordpress.com/2014/07/16/integrating-gyros-2/
#https://engineersportal.com/blog/2017/9/25/accelerometer-on-an-elevator

#DEPTH CAMERA INFO
#1. https://www.mouser.co.uk/new/Intel/intel-realsense-camera-400/
#Scatter Plot: pyqt + opengl scatter plot

#MOCAP VISUALIZATION
#1. Using Motion Builder: http://awforsythe.com/tutorials/pyfbsdk-1
#2. 

imu_num = 1

In [ ]:
def server():
    #initialize the server
    #start server
    #send data continously to port
    return 0

def client():
    #initialize client
    #start client
    #receive data from port
    return 0

In [ ]:
def ypr():
    
    return 0


def plotYPR(array):
    t = linspace(0, 2*math.pi, 400)
    a = sin(t)
    b = cos(t)
    c = a + b

    plt.plot( a, 'r') # plotting t, a separately 
    plt.plot( b, 'b') # plotting t, b separately 
    plt.plot( c, 'g') # plotting t, c separately 
    plt.show()
    return 0

In [ ]:
#Create a file in 'h5py' file and save list continously when called
#Arguments: filename (string)
##Check the storing of "TIME STAMP" data ==> there is some problem with it
def createFile(filename):
    flist=[]
    try:
        file = h5py.File(filename, "w")
    except:
        print("Error: Creating dataset file")
        file.close()

    for i in range(imu_num):
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/acc", (0,3), maxshape=(None,3), dtype='i'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/gyro",(0,3), maxshape=(None,3), dtype='i'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/mag", (0,3), maxshape=(None,3), dtype='i'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/time",(0,1), maxshape=(None,1), dtype='float32'))

    return file, flist

def saveData(flist, connection, imu_data, tstamp):
    data = np.array([np.array(xi) for xi in imu_data[connection]], 'int8')#.T #take care of unsigned to signed int conversion
    cur_size = len(flist[connection])       #current row size of h5 file
    data_size= len(imu_data[connection])    #incoming data row size
    
    for i in range(3):
        flist[connection*4 + i].resize((cur_size + data_size, 3))                         #extending h5 row size
        flist[connection*4 + i][cur_size: cur_size + data_size] = data[:, 3*i:3*i+3]      #data[3*i : 3*i + 3].T   #writing imu data to corresponding subgroup
        
    time = np.array(tstamp[connection], dtype=np.float32)                                 #converting time stamp data to numpy array (float)
    flist[connection*4 + 3].resize((cur_size + data_size, 1))                             #resizing the file dataset for storing
    flist[connection*4 + 3][cur_size: cur_size + data_size] = time.reshape((data_size,1)) #finally storing time stamp data
    
    return 0

def loadData(d):
    return 0

def closeFile(file):
    file.close()
    return 0

In [ ]:
connect_msg = []
connect_msg.append(b'\x00\x0f\x06\x03\x2c\x58\x0f\xee\x4f\x98\x00\x3c\x00\x4c\x00\x64\x00\x00\x00')
connect_msg.append(b'\x00\x0f\x06\x03\x8d\xbb\x35\x30\x30\xdd\x01\x3c\x00\x4c\x00\x64\x00\x00\x00')
connect_msg.append(b'\x00\x0f\x06\x03\xdc\x81\x3a\x5d\xf1\xf3\x01\x3c\x00\x4c\x00\x64\x00\x00\x00')
connect_msg.append(b'\x00\x0f\x06\x03\xed\x9a\x74\xb3\x71\xf8\x01\x3c\x00\x4c\x00\x64\x00\x00\x00')

#ble_cmd_attclient_attribute_write
#ONLY handle is different => Something can be done
att_write_msg = []
att_write_msg.append(b'\x00\x06\x04\x05\x00\x0c\x00\x02\x01\x00')
att_write_msg.append(b'\x00\x06\x04\x05\x00\x29\x00\x02\x01\x00')
att_write_msg.append(b'\x00\x06\x04\x05\x01\x29\x00\x02\x01\x00')
att_write_msg.append(b'\x00\x06\x04\x05\x02\x29\x00\x02\x01\x00')

#BLE setting freq to 50 Hz ==> need to varify
#ONLY handle is different so ==> Something can be done
set_freq_msg=[]
set_freq_msg.append(b'\x00\x05\x04\x05\x00\x22\x00\x01\x32')
set_freq_msg.append(b'\x00\x05\x04\x05\x01\x22\x00\x01\x32')
set_freq_msg.append(b'\x00\x05\x04\x05\x02\x22\x00\x01\x32')

def connect_imu(port):
    #################################################################
    #Set Frequency of data
    #Dynamically store connection handle corresponding to BLE address
    #Use that dynamic connection handle for all the further operation
    #It can be scanned to see if the known devices are present ON
    #If ON then connect to the device
    #################################################################
    ser = serial.Serial(
        port = port,
        baudrate=115200,
        parity =serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.EIGHTBITS
    )
    print(ser.isOpen())
    ser.flushInput()
    ser.flushOutput()
    #4-5 => result => 0: success else error for non-zero
    #6 => connection handle ==> useful for multiple connections
    for i in range(imu_num):
        ser.write(connect_msg[i])    #connecting 1st BLE
        s = ser.read(27)
        print ("response:", binascii.hexlify(bytearray(s)))
        time.sleep(1)
        
    #Attribute Write command can be up to 20 bytes: #4=> connection handle, 5-6 => result => 0 for success else error
    #Reply with== a. ble_rsp_attclient_attribute_write => 7 bytes
    #b. ble_evt_attclient_procedure_completed => 9 bytes
    
    #for i in range(imu_num):
    #    ser.write(set_freq_msg[i])
    #    time.sleep(1)
    #   s = ser.read(7+9)
    #    print ("response:", binascii.hexlify(bytearray(s)))
        
    for i in range(imu_num):
        ser.write(att_write_msg[i])
        time.sleep(1)
        s = ser.read(7+9)
        print ("response:", binascii.hexlify(bytearray(s)))
        
    return ser


#Basic plotting function of IMUs data, currently IMU is plotted
#IMU_DATA format: [ax,ay,az,gx,gy,gz,mx,my,mz,tstamp]
def plot(plot_data, imu_num, tstamp, connection):
    global imu_data_chunk
    display.clear_output(wait=True)
    print("Average Sampling Freq. of", imu_data_chunk, "elements is ::", getFreq(tstamp, connection, show=False))
    f, axarr = plt.subplots(3, imu_num, figsize=(15,12))
    if (imu_num>1):
        for i in range(imu_num):
            #converting list of list to 2-D array in form of "signed" 8-bit integer
            if(len(imu_data[i])==0): continue   #if in case some BLE has empty data array 
            data = np.array([np.array(xi) for xi in plot_data[i]], 'int8').T  
            axarr[0, i].plot(data[0])
            axarr[1, i].plot(data[1])
            axarr[2, i].plot(data[2])
    if (imu_num==1):
        for i in range(imu_num):
            #converting list of list to 2-D array in form of "signed" 8-bit integer
            if(len(imu_data[i])==0): continue   #if in case some BLE has empty data array 
            data = np.array([np.array(xi) for xi in imu_data[i]], 'int8').T  
            axarr[0].plot(data[0])
            axarr[1].plot(data[1])
            axarr[2].plot(data[2])
    else:
        print("Invalid total number of IMUs available")
        return 0
    
    plt.show()
    display.display(pl.gcf())
    return 0

imu_data_chunk = 300
def getFreq(tstamp, connection, show=False):
    global imu_data_chunk
    tm = np.array(tstamp[connection])
    dt = tm[1:len(tm)-1] - tm[0:len(tm)-2]
    freq = 1./dt
    if show:
        print("Individual Sampling Freq.:: ", freq,"\n")
    avg_freq = len(dt)/np.sum(dt)
    #print("Average Sampling Freq. of", imu_data_chunk, "elements is ::", avg_freq)
    return avg_freq

temp_str    = []
plot_count  = 0
debug       = False

bgapi_rx_buffer = []
bgapi_rx_expected_length = 0
imu_data  = deque([deque([]) for _ in range(imu_num)])
tstamp    = deque([deque([]) for _ in range(imu_num)])
plot_data = deque([deque([]) for _ in range(imu_num)])
def parse(b, flist, isPlot=False, isCurie=False):
    global bgapi_rx_buffer, bgapi_rx_expected_length, temp_str, imu_data, imu_num, plot_count, imu_data_chunk, tstamp
    t=time.time()
    if len(bgapi_rx_buffer) == 0 and (b==0x80): #(b == 0x00 or b == 0x80 or b == 0x08 or b == 0x88):
        bgapi_rx_buffer.append(b)
    elif len(bgapi_rx_buffer) == 1:
        bgapi_rx_buffer.append(b)
        bgapi_rx_expected_length = 4 + (bgapi_rx_buffer[0] & 0x07) + bgapi_rx_buffer[1]
    elif len(bgapi_rx_buffer) > 1:
        bgapi_rx_buffer.append(b)
        
    if bgapi_rx_expected_length > 0 and len(bgapi_rx_buffer) == bgapi_rx_expected_length:
        if debug: print ('<=[ ' + ' '.join(['%02X' % b for b in bgapi_rx_buffer ]) + ' ]')
        packet_type, payload_length, packet_class, packet_command = bgapi_rx_buffer[:4]
        #bgapi_rx_payload = b''.join(chr(i) for i in bgapi_rx_buffer[4:])
        bgapi_rx_payload=bgapi_rx_buffer[4:]
        temp_str=bgapi_rx_buffer
        bgapi_rx_buffer = []
        
        #Processing part can be further moved to some other function
        if packet_type & 0x88 == 0x80:
            if packet_command == 5: # ble_evt_attclient_attribute_value
                #connection, atthandle, type, value_len = struct.unpack('<BHBB', bgapi_rx_payload[:5])
                #value_data = [ord(b) for b in bgapi_rx_payload[5:]]
                connection = bgapi_rx_payload[0]
                value_data = bgapi_rx_payload[5:]
                if((connection < imu_num)):                           #[bgapi_rx_payload[0]] = connection
                    tstamp[connection].append(time.time())            #storing time stamp data
                    if isCurie:
                        yaw  = (((value_data[1] << 8) | value_data[0])/100 - 180)
                        pitch= (((value_data[3] << 8) | value_data[2])/100 - 180)
                        roll = (((value_data[5] << 8) | value_data[4])/100 - 180)
                        return yaw, pitch, roll
                    else:
                        imu_data[connection].append(value_data[2::2])     #storing all 9-axis imu data
                        if(len(imu_data[connection]) == imu_data_chunk):
                            saveData(flist, connection, imu_data, tstamp)
                            print("Average Sampling Freq. of", imu_data_chunk, "elements is ::", getFreq(tstamp, connection, show=False))     #imu_data_chunk/(tstamp[connection][0]-tstamp[connection][299])
                            imu_data[connection].clear()
                            tstamp[connection].clear()
                        if isPlot:
                            plot_data[connection].append(value_data[2::2])       #using numpy slicing for every second element
                            if(len(plot_data[connection])> 300):
                                plot_data[connection].popleft()                  #Write data to list i.e. append data and pop
                                plot_count=plot_count+1
                                if(plot_count>300):
                                    plot(plot_data, imu_num, tstamp, connection) #Only the current graph can be sent for update
                                    plot_count=0
                        return value_data[2:8:2], value_data[8:14:2], value_data[14:20:2]

                return 0
            else:
                bgapi_rx_buffer=[]
                return 0
                
        else:
            bgapi_rx_buffer=[]
            return 0

In [ ]:
#file.close()

##THE SYSTEM IS LIKE A CASCADED AS OF NOW ###
#take a frame and continuously pass to the object detection module
#while(detection is less than threshold)
#    frame = cv2.VideoCapture(0)
#    if frame.isOpened: print("open")
#    scene = frame.grab()
    #pass the frame to detection
    #increment the counter with every detection
    #decrement if interested object not present 
    ### can be decreased only to a base value
    #Once reaching a threshold then break and trigger the arm activation motion function

###Both IMUs are assumed to be connected to the PC with BLE and there is a direct flow of data
#Switch to arm motion function
#==>decide a segmentation window and pass down the IMU value to the classifier in the window
#Once done the see the output of classifer and set the intended output

imgcv = cv2.imread("yolo/bottle.jpg")
result = tfnet.return_predict(imgcv)
print(result)

tmpp = []
yprData = deque([])
tmp_count = 0
tmp_count1= 0

file, flist = createFile('11testfile.h5')
ser=connect_imu('/dev/ttyACM0')
while (1):
    try:
        while (ser.inWaiting()):
            temp=ord(ser.read())
            tmpp = parse(temp, flist, isPlot=False, isCurie=True);
            if((tmpp!=None) & (tmpp!=0)):
                yaw  = np.array(tmpp[0])
                pitch= np.array(tmpp[1])
                roll = np.array(tmpp[2])
                yprData.append([yaw, pitch, roll])
                tmp_count = tmp_count + 1
                if(tmp_count>300):
                    #yprData.popleft()
                    tmp_count1 = tmp_count1 + 1
                    if(tmp_count1>30):
                        display.clear_output()
                        fig=plt.figure(figsize=(18, 10), dpi= 80, facecolor='w', edgecolor='k')
                        a=np.array(yprData).T
                        plt.plot(a[0][len(yprData)-300:len(yprData)], 'r')
                        plt.plot(a[1][len(yprData)-300:len(yprData)], 'g')
                        plt.plot(a[2][len(yprData)-300:len(yprData)], 'b')
                        plt.show()
                        tmp_count1=0
    except KeyboardInterrupt:
        np.savetxt("ypr.txt", yprData, newline=" ")
        ser.write(b'\x00\x01\x03\x00\x00')
        file.close()
        ser.flushInput()
        ser.flushOutput()
        ser.close()

In [ ]:
getFreq(tstamp, 0, show=False)

In [ ]:
#file.close()

##check gyro const
gyroConst = 250/32768.0 #the gyro max. degrees per sec. is 250 which corrosponds to 128 ADC vice versa -128 corresponds to -250
sampPeriod= 1/30    #inverse of sampling frequency
tmpp = []
yprData = deque([])
tmp_count = 0
tmp_count1= 0

ren, renWin = skeleton.setRenWindow()
sphereActor, torso, upperArmL, upperArmR, foreArmL, foreArmR = skeleton.makeSkeleton(ren)
skeleton.showAxes(ren, upperArmR)             #show axes
skeleton.setCamera(ren)
iren = skeleton.addInteractor(renWin)
iren.Initialize()                             #enable user interface interactor

heading = madgwickahrs.MadgwickAHRS(sampleperiod=1/30)
ser=connect_imu('/dev/ttyACM0')
file, flist = createFile('11testfile.h5')
while (1):
    try:
        #ble_evt_attclient_attribute_value({ 'connection': connection, 'atthandle': atthandle, 'type': type, 'value':
        while (ser.inWaiting()):
            temp=ord(ser.read())
            tmpp = parse(temp, flist, isPlot=False, isCurie=False);
            if((tmpp!=None) & (tmpp!=0)):
                heading.update(np.array(tmpp[1], 'int8')*gyroConst , np.array(tmpp[0], 'int8'), np.array(tmpp[2], 'int8'))
                ahrs = heading.quaternion.to_euler_angles()
                roll = ahrs[0]*180/3.14
                pitch = ahrs[1]*180/3.14
                yaw = ahrs[2]*180/3.14
                
                upperArmR.RotateX(roll)
                upperArmR.RotateY(pitch)
                upperArmR.RotateZ(yaw)
                renWin.Render()
                
                yprData.append([yaw, pitch, roll])
                tmp_count = tmp_count + 1
                if(tmp_count>300):
                    yprData.popleft()
                    tmp_count1 = tmp_count1 + 1
                    if(tmp_count1>30):
                        display.clear_output()
                        fig=plt.figure(figsize=(18, 10), dpi= 80, facecolor='w', edgecolor='k')
                        a=np.array(yprData).T
                        plt.plot(a[0], 'r')
                        plt.plot(a[1], 'g')
                        plt.plot(a[2], 'b')
                        plt.show()
                        tmp_count1=0
                        #plt.pause(0.05)
                #print(yaw,pitch,roll)
    except KeyboardInterrupt:
        ser.write(b'\x00\x01\x03\x00\x00')
        ser.write(b'\x00\x01\x03\x00\x01')
        ser.write(b'\x00\x01\x03\x00\x02')
        ser.flushInput()
        ser.flushOutput()
        file.close()
        ser.close()

In [ ]:
#arr[arr > 255] = x

#dataset="2016-06-02--21-39-29"
#log = h5py.File("dataset/log/"+dataset+".h5", "r")
#cam = h5py.File("dataset/camera/"+dataset+".h5", "r")
#img = np.array((cam['X'][log['cam1_ptr'][200000]]))
#predicted_steers = model1.predict(img[None, :, :, :])

#print(temp_str)
#list(chr(i) for i in temp_str[4:])

#bgapi_rx_payload = b''.join(list(chr(i) for i in temp_str[4:]))
#y=b"128, 24, 4, 5, 0, 40, 0, 1, 19, 0, 255, 0, 255, 7, 255, 190, 255, 0, 255, 255, 255, 0, 134, 90, 175, 14, 174, 145"
#connection, atthandle, type, value_len = struct.unpack('<BHBB', y[:5])
#print(value_len)

#data_to_write1 = np.random.random(size=(100,20)) # or some such

#f = h5py.File('temp7.h5','a')
#temp=f.create_dataset("IMU", (9,))

#with h5py.File('name-of-file1.h5', 'a') as hf:
#    hf.create_dataset("name-of-dataset",  data=data_to_write)
#a = [[] for _ in range(3)]

#np.random.random((N,N))

##using subplots
#plt.figure(num=None, figsize=(14, 11), dpi=100, facecolor='w', edgecolor='k')
#plt.subplot(3,1,1)
#plt.plot(ax1)
#plt.subplot(3,1,2)
#plt.plot(ay1)
#plt.subplot(3,1,3)
#plt.plot(az1)
#plt.show()
#display.display(pl.gcf())

#f = h5py.File('vlen.h5','w')
#dt = h5py.special_dtype(vlen=np.dtype('float64'))
#dataset = f.create_dataset('vlen',(100,), dtype=dt)

#axarr[0, i].plot(np.array(ax[i], 'int8'))
#axarr[1, i].plot(np.array(ay[i], 'int8'))
#axarr[2, i].plot(np.array(az[i], 'int8'))
        
##can be made more shortened
#ax=deque([deque([]) for _ in range(imu_num)])
#ay=deque([deque([]) for _ in range(imu_num)])
#az=deque([deque([]) for _ in range(imu_num)])
#ax[connection].append(value_data[2])
#ay[connection].append(value_data[4])
#az[connection].append(value_data[6])
#ax[connection].popleft()
#ay[connection].popleft()
#az[connection].popleft()

In [ ]:
float c1 = cos(radians(roll))
float s1 = sin(radians(roll))
float c2 = cos(radians(pitch))
float s2 = sin(radians(pitch))
float c3 = cos(radians(yaw))
float s3 = sin(radians(yaw))
applyMatrix (c2*c3,    s1*s3+c1*c3*s2, c3*s1*s2-c1*s3, 0,
            -s2,       c1*c2,          c2*s1,          0,
             c2*s3,    c1*s2*s3-c3*s1, c1*c3+s1*s2*s3, 0,
             0,        0,              0,              1);